# MOE 11.3 Producer Consumer Model Scenario

## Diana McSpadden (hdm5s)

## Shared Resource Description

The shared resource is the **laundry hamper**.

* 5 humans are depositing items of dirty laundry in the hamper
* The hamper holds 100 items of dirty laundry, although the hamper can get crammed with up to 105 items. A warning message is displayed if the ha,per has over 99 items (for this toy problem, if the hamper is full, the humans must just keep wearing dirty clothes).
* Laundry loads are only performed when there are 20 items of dirty laundry, and a load only cleans 20 items of dirty laundry.
* There are never negative numbers of dirty laundry items.
* While there are **5** humans creating dirty laundry, there is only **1** washing machine to clean the laundry. But the washing machine can be run several times a day (days and times are simulated with time.sleep())


### Implementation:

* My toy problem lets the humans add a random number of dirty items between 1 and 10 dirty items
* And, while ideally, a full laundry load has exactly 20 items and the condition is only met when more than 20 items are available for a load, I allow some "human" error and laundry loads clean a random number of items between 18 and 20.
* And, ideally, the full hamper could only hold 100 items, it can hold up to 102 items (and more may be piled on because of random number of dirty items that can be created by the "people") and gives a warning if the number of items is over 99.


In [1]:
# include the libraries needed
from threading import Thread
import threading
import time
import random # for random number of items to add to laundry hamper and to clean in a laundy load

In [2]:
class LaundryHamper:
  def __init__(self):
    self.lock = threading.Lock()
    self.laundryLoadCondition = threading.Condition(self.lock)
    self.dirtyItemCount = 0

  def addDirtyItems(self, humanName):
    self.lock.acquire()
    try:
      if (self.dirtyItemCount > 99):
        print("WARNING: The hamper is full.")
        
      # only add items if the laundry hamper has fewer than 100 items
      while (self.dirtyItemCount >= 102):
        self.laundryLoadCondition.wait()

      # determine how many dirty items this human will add: between 1 and 9
      numberOfDirtyItems = random.randint(1,10)
      # add the dirty items to the hamper
      newCount = self.dirtyItemCount + numberOfDirtyItems
      self.dirtyItemCount = newCount

      # print the current hamper balance and who added items
      print("%s added %d items to the hamper. Number of dirty items in hamper is: %d" % (humanName, numberOfDirtyItems, newCount))
      # notify the other threads
      self.laundryLoadCondition.notifyAll()
    except:
      print("The laundry hamper broke.")
    finally:
      self.lock.release()
  
  def cleanLaundry(self):
    self.lock.acquire()
    try:
      # wait until there are 20 dirty laundry items to do a load of laundry
      while (self.dirtyItemCount < 20):
        self.laundryLoadCondition.wait()

      # determine how many dirty items will be cleaned
      numberOfItemsToClean = random.randint(18,20)
      # remove the items from the hamper
      newCount = self.dirtyItemCount - numberOfItemsToClean
      self.dirtyItemCount = newCount

      # print the current hamper balance after cleaning
      print("A load of laundry with %d items was completed. Number of dirty items in hamper is: %d" % (numberOfItemsToClean, newCount))
      # notify the other threads
      self.laundryLoadCondition.notifyAll()

    except:
      print("The washing machine broke.")
    finally:
      self.lock.release()
  
  def getNumberOfHamperItems(self):
    return self.dirtyItemCount

def triggerAddingDirtyLaundry(hamper, humanName, days):
  for i in range(days):
    hamper.addDirtyItems(humanName)
    # add a little time between adding dirty laundry
    time.sleep(1)

def triggerCleanLaundry(hamper, numLoads):
  for i in range(numLoads):
    hamper.cleanLaundry()
    # add a little time between laundry loads
    time.sleep(4)


if __name__ == '__main__':

    hamper = LaundryHamper()
    human1 = "Buffy"
    human2 = "Willow"
    human3 = "Xander"
    human4 = "Tara"
    human5 = "Giles"

    t_h1 = Thread(target=triggerAddingDirtyLaundry, args=(hamper, human1, 14,))
    t_h2 = Thread(target=triggerAddingDirtyLaundry, args=(hamper, human2, 14,))
    t_h3 = Thread(target=triggerAddingDirtyLaundry, args=(hamper, human3, 14,))
    t_h4 = Thread(target=triggerAddingDirtyLaundry, args=(hamper, human4, 14,))
    t_h5 = Thread(target=triggerAddingDirtyLaundry, args=(hamper, human5, 14,))

    t_clean1 = Thread(target=triggerCleanLaundry, args=(hamper, 3,))
    t_clean2 = Thread(target=triggerCleanLaundry, args=(hamper, 3,))
    t_clean3 = Thread(target=triggerCleanLaundry, args=(hamper, 3,))
    t_clean4 = Thread(target=triggerCleanLaundry, args=(hamper, 3,))
    t_clean5 = Thread(target=triggerCleanLaundry, args=(hamper, 3,))

    t_clean1.start()
    t_h1.start()
    t_h2.start()
    t_clean2.start()
    t_h3.start()
    t_clean3.start()
    t_clean4.start()
    t_h4.start()
    t_h5.start()
    t_clean5.start()
    
    #print("Laundry Example Finished.")

Buffy added 9 items to the hamper. Number of dirty items in hamper is: 9
Willow added 3 items to the hamper. Number of dirty items in hamper is: 12
Xander added 8 items to the hamper. Number of dirty items in hamper is: 20
A load of laundry with 18 items was completed. Number of dirty items in hamper is: 2
Tara added 9 items to the hamper. Number of dirty items in hamper is: 11
Giles added 7 items to the hamper. Number of dirty items in hamper is: 18
Willow added 10 items to the hamper. Number of dirty items in hamper is: 28
A load of laundry with 20 items was completed. Number of dirty items in hamper is: 8
Buffy added 7 items to the hamper. Number of dirty items in hamper is: 15
Xander added 8 items to the hamper. Number of dirty items in hamper is: 23
A load of laundry with 20 items was completed. Number of dirty items in hamper is: 3
Tara added 2 items to the hamper. Number of dirty items in hamper is: 5
Giles added 3 items to the hamper. Number of dirty items in hamper is: 8
Willo